# * VINSIGHT : Data Monitoring
    Process required -> "Import-AGG_PERF_NEWCO_SNAP"

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

In [2]:
# Input parameter

op_dir = 'data'
op_monthly_file = f'monthly_snap_{str_curr_dt}'
op_daily_file = f'daily_snap_{str_curr_dt}'

print(f"\nParameter input...\n\n   -> op_dir: {op_dir}\n   -> op_monthly_file: {op_monthly_file}\n   -> op_daily_file: {op_daily_file}")


Parameter input...

   -> op_dir: data
   -> op_monthly_file: monthly_snap_20240624
   -> op_daily_file: daily_snap_20240624


## Create DataFrame
    DB -> Output file

    Source : AUTOKPI.AGG_PERF_NEWCO_SNAP

In [3]:
# Connect : AKPIPRD
dsn = f'{AKPIPRD_user}/{AKPIPRD_pwd}@{AKPIPRD_host}:{AKPIPRD_port}/{AKPIPRD_db}'
conn = oracledb.connect(dsn)
print(f'\n{AKPIPRD_db} : Connected\n\nProcessing...')
cur = conn.cursor()


query = """
    /*** Monthly Snap ***/

    SELECT TM_KEY_MTH, CENTER, METRIC_GRP, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, CHANNEL_CD, AGG_TYPE, UOM
        , CAST(SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_TMP END) AS DECIMAL(18,2)) AS P_ACTUAL
        , CAST(SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_TMP END) AS DECIMAL(18,2)) AS G_ACTUAL
        , CAST(SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_TMP END) AS DECIMAL(18,2)) AS H_ACTUAL
        , CAST(SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_TMP END) AS DECIMAL(18,2)) AS HH_ACTUAL
        , CAST(SUM(CASE WHEN AREA_TYPE = 'P' THEN TARGET_TMP END) AS DECIMAL(18,2)) AS P_TARGET
        , CAST(SUM(CASE WHEN AREA_TYPE = 'G' THEN TARGET_TMP END) AS DECIMAL(18,2)) AS G_TARGET
        , CAST(SUM(CASE WHEN AREA_TYPE = 'H' THEN TARGET_TMP END) AS DECIMAL(18,2)) AS H_TARGET
        , CAST(SUM(CASE WHEN AREA_TYPE = 'HH' THEN TARGET_TMP END) AS DECIMAL(18,2)) AS HH_TARGET
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF, MIN(TM_KEY_DAY) MIN_DAY, MAX(TM_KEY_DAY) MAX_DAY, MAX(PPN_TM) PPN_TM, MAX(LOAD_DATE) LOAD_DATE
        
    FROM (
        SELECT TM_KEY_YR, TM_KEY_QTR, TM_KEY_MTH, TM_KEY_WK, TM_KEY_DAY
            , CENTER, PRODUCT_GRP, COMP_CD, METRIC_GRP, METRIC_CD, METRIC_NAME, SEQ, ACTUAL_AS_OF, AGG_TYPE, RR_IND, GRY_IND, UOM, AREA_TYPE, AREA_CD, AREA_NAME
            , CASE 	WHEN AGG_TYPE = 'S' THEN ACTUAL_SNAP 
                    ELSE (CASE WHEN TM_KEY_DAY = MAX(TM_KEY_DAY) OVER(PARTITION BY METRIC_CD, TM_KEY_MTH) THEN ACTUAL_AGG END)
                    END ACTUAL_TMP
            , CASE 	WHEN AGG_TYPE = 'S' THEN TARGET_SNAP 
                    ELSE (CASE WHEN TM_KEY_DAY = MAX(TM_KEY_DAY) OVER(PARTITION BY METRIC_CD, TM_KEY_MTH) THEN TARGET_AGG END)
                    END TARGET_TMP
            , ACTUAL_SNAP, TARGET_SNAP, BASELINE_SNAP, ACTUAL_AGG, TARGET_AGG, BASELINE_AGG, PPN_TM, LOAD_DATE
            , CASE WHEN REGEXP_LIKE(METRIC_CD, '[0-9]A[A-K]$') THEN SUBSTR(METRIC_CD,-2) ELSE 'ALL' END CHANNEL_CD
        FROM AUTOKPI.AGG_PERF_NEWCO_SNAP NOLOCK
    ) MTH_SNAP

    --WHERE CHANNEL_CD = 'ALL'
    GROUP BY TM_KEY_MTH, CENTER, METRIC_GRP, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME, CHANNEL_CD, AGG_TYPE, UOM
    ORDER BY TM_KEY_MTH, CENTER, METRIC_GRP, PRODUCT_GRP, COMP_CD, METRIC_CD
"""


try:
    # Create Dataframe
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')
    cur.execute(query)
    rows = cur.fetchall()
    df = pd.DataFrame.from_records(rows, columns=[x[0] for x in cur.description])
    print(f'\n   -> DataFrame : {df.shape[0]} rows, {df.shape[1]} columns')

    # Generate CSV file
    df.to_csv(f'{op_dir}/{op_monthly_file}.csv', index=False, encoding='utf-8')
    print(f'\n   -> Generate "{op_monthly_file}.csv" successfully')

    # # Generate Excel file
    # df.to_excel(f'{op_dir}/{op_monthly_file}.xlsx', sheet_name='Data', index=False)
    # print(f'\n   -> Generate "{op_monthly_file}.xlsx" successfully')


except oracledb.DatabaseError as e:
    print(f'Error with Oracle : {e}')


finally:
    cur.close()
    conn.close()
    print(f'\n{AKPIPRD_db} : Disconnected')



AKPIPRD : Connected

Processing...

   -> Execute query... 2024-06-24, 18:14:55

   -> DataFrame : 7272 rows, 23 columns

   -> Generate "monthly_snap_20240624.csv" successfully

AKPIPRD : Disconnected


In [4]:
df.tail(3)

,TM_KEY_MTH,CENTER,METRIC_GRP,PRODUCT_GRP,COMP_CD,METRIC_CD,METRIC_NAME,CHANNEL_CD,AGG_TYPE,UOM,...,HH_ACTUAL,P_TARGET,G_TARGET,H_TARGET,HH_TARGET,ACTUAL_AS_OF,MIN_DAY,MAX_DAY,PPN_TM,LOAD_DATE
7269,202406,Sales,Sales,TVS,TRUE,TB4R001000,TVS Inflow M1,ALL,S,baht,...,5242.0,NaN,NaN,NaN,NaN,20240618.0,20240601,20240620,2024-06-21 12:37:25,2024-06-21 15:59:30.795783
7270,202406,Sales,Sales,TVS,TRUE,TB4R001000CORP,TVS Inflow M1,ALL,S,baht,...,5242.0,NaN,NaN,NaN,NaN,20240618.0,20240601,20240620,2024-06-21 12:37:25,2024-06-21 15:59:30.795783
7271,202406,Sales,Sales,TVS,TRUE,TB4R001600,TVS GA ARPU,ALL,N,baht,...,3728.0,NaN,NaN,NaN,NaN,20240619.0,20240601,20240620,2024-06-21 12:37:25,2024-06-21 15:59:30.795783


## Reconcile

In [7]:
''' Function '''

def view_group(grp, cd, name):
    grp = grp
    cd = cd
    name = name
    flag = ''

    # CORP & MCOM
    if re.search('C$|H$|MCOM$', cd) and (not re.search('A[A-K]$', cd)): flag = 'CORP & MCOM'
    elif re.search('CUS$', cd): flag = 'Cust Location'
    # Revenue
    elif grp == 'Revenue' and any(x in name for x in ['New Revenue', 'Existing Revenue']): flag = 'New/Existing'
    elif grp == 'Revenue' and any(x in name for x in ['Paid Amount', 'On Due', 'Overdue']): flag = 'Paid Amount'
    elif grp == 'Revenue' and any(x in name for x in ['Revenue']): flag = 'Revenue'
    # Sales
    elif grp == 'Sales' and any(x in name for x in ['Inflow M1']): flag = 'Inflow M1'
    elif grp == 'Sales' and any(x in name for x in ['Inflow M2']): flag = 'Inflow M2'
    elif grp == 'Sales' and any(x in name for x in ['Gross Add']): flag = 'Gross Adds'
    elif grp == 'Sales' and any(x in name for x in ['%AP']): flag = '%AP'
    elif grp == 'Sales' and any(x in name for x in ['AP 1D']): flag = 'AP 1D'
    elif grp == 'Sales' and any(x in name for x in ['AP In Month']): flag = 'AP MTH'
    elif grp == 'Sales' and any(x in name for x in ['Activation Subs']): flag = 'Activation'
    elif grp == 'Sales' and any(x in name for x in ['Conversion']): flag = '%Conversion'
    elif grp == 'Sales' and any(x in name for x in ['GA ARPU', 'GA RC']): flag = 'GA ARPU/RC'
    # Subs
    elif grp == 'Subs' and any(x in name for x in ['Net Add']): flag = 'Net Adds'
    elif grp == 'Subs' and any(x in name for x in ['%NAD']): flag = '%NAD'
    elif grp == 'Subs' and any(x in name for x in ['%M4']): flag = '%M4'
    elif grp == 'Subs' and any(x in name for x in ['Reported Sub']): flag = 'Reported Subs'
    elif grp == 'Subs' and any(x in name for x in ['Usage Subs', 'Active Caller', 'Active Subs']): flag = 'Active Subs'
    elif grp == 'Subs' and any(x in name for x in ['NAD']): flag = 'NAD'
    elif grp == 'Subs' and any(x in name for x in ['Revenue Subs']): flag = 'Rev Subs'
    # MKS
    elif grp == 'Market Share' and any(x in name for x in ['Broadband']): flag = '%BB MKS'
    elif grp == 'Market Share' and (not any(x in name for x in ['Broadband'])) & any(x in name for x in ['(Subs)']): flag = 'MB MKS(Subs)'
    elif grp == 'Market Share' and (not any(x in name for x in ['Broadband', '(Subs)'])): flag = '%MB MKS'
    # Churn
    elif grp == 'Retention & Churn' and any(x in name for x in ['Churn Subs']): flag = 'Churn Subs'
    elif grp == 'Retention & Churn' and any(x in name for x in ['Churn Rate']): flag = '%Churn Rate'
    # Others
    elif any(x in name for x in ['ARPU']): flag = 'ARPU'
    elif any(x in name for x in ['SubBase']): flag = 'SubBase'
    elif any(x in name for x in ['New Subs']): flag = 'New Subs'
    elif any(x in name for x in ['Silent']): flag = 'Silent'
    elif any(x in name for x in ['60DPD']): flag = '60DPD'
    elif any(x in name for x in ['Quality']): flag = 'Quality'
    else: flag = 'Unknown'
 
    return flag

In [8]:
''' Rawdata '''

data_src = f'{op_dir}/{op_monthly_file}.csv'
raw_df = pd.read_csv(data_src, low_memory=False)
print(f'\nraw_df : {raw_df.shape[0]} rows, {raw_df.shape[1]} columns')
# raw_df.tail(3)


raw_df : 7272 rows, 23 columns


In [9]:
''' Create Temp DataFrame '''

''' Add columns '''
tmp_df = raw_df
tmp_df['TMP_CD'] = tmp_df['METRIC_CD'].replace(r'AA$|AB$|AC$|AD$|AE$|AF$|AG$|AH$|AI$|AJ$|AK$', '', regex=True)
tmp_df['TMP_NAME'] = tmp_df['METRIC_NAME'].replace(r' : Account Executive| : B2B| : Branded Retail| : Contact Center| : Direct Sales| : Key Account| : Modern Trade| : Others| : Own Digital| : Retail Sales| : Wholesales', '', regex=True)
tmp_df['VIEW_GRP'] = tmp_df.apply(lambda x: view_group(str(x['METRIC_GRP']), str(x['METRIC_CD']), str(x['METRIC_NAME'])), axis=1)
# tmp_df['CDS_FLAG'] = np.where(tmp_df['METRIC_CD'].isin(['TB1R000109', 'TB1S000109', 'TB1R000106', 'TB1S000106', 'TB1S000102', 'TB1S000103', 'TB1S000104', 'TB0R00010002', 'TB2S000100', 'TB1S000101', 'TB3S000100', 'TB3S000101', 'TB3S000102', 'TB3S000103', 'TB3S000900', 'TB3S000901', 'TB3S000902', 'TB4S000100', 'TB4S000104', 'TB4S001300', 'TB0R00010001', 'TB2R000500', 'TB1R000900', 'TB3R000600', 'TB3R000601', 'TB3R000602', 'TB4R001000', 'TB1R001000', 'TB4S001400', 'TB4R001700']), 'Y', 'N')
tmp_df['CDS_FLAG'] = np.where((tmp_df['METRIC_CD'].str.contains('^TB1R000109|^TB1S000109|^TB1R000106|^TB1S000106|^TB1S000102|^TB1S000103|^TB1S000104|^TB0R00010002|^TB2S000100|^TB1S000101|^TB3S000100|^TB3S000101|^TB3S000102|^TB3S000103|^TB3S000900|^TB3S000901|^TB3S000902|^TB4S000100|^TB4S000104|^TB4S001300|^TB0R00010001|^TB2R000500|^TB1R000900|^TB3R000600|^TB3R000601|^TB3R000602|^TB4R001000|^TB1R001000|^TB4S001400|^TB4R001700')) & (tmp_df['METRIC_CD'].str.contains('[0-9]$|[0-9]A[A-K]$')), 'Y', 'N')

tmp_df.tail(3)

,TM_KEY_MTH,CENTER,METRIC_GRP,PRODUCT_GRP,COMP_CD,METRIC_CD,METRIC_NAME,CHANNEL_CD,AGG_TYPE,UOM,...,HH_TARGET,ACTUAL_AS_OF,MIN_DAY,MAX_DAY,PPN_TM,LOAD_DATE,TMP_CD,TMP_NAME,VIEW_GRP,CDS_FLAG
7269,202406,Sales,Sales,TVS,TRUE,TB4R001000,TVS Inflow M1,ALL,S,baht,...,NaN,20240618.0,20240601,20240620,2024-06-21 12:37:25,2024-06-21 15:59:30.795783,TB4R001000,TVS Inflow M1,Inflow M1,Y
7270,202406,Sales,Sales,TVS,TRUE,TB4R001000CORP,TVS Inflow M1,ALL,S,baht,...,NaN,20240618.0,20240601,20240620,2024-06-21 12:37:25,2024-06-21 15:59:30.795783,TB4R001000CORP,TVS Inflow M1,Inflow M1,N
7271,202406,Sales,Sales,TVS,TRUE,TB4R001600,TVS GA ARPU,ALL,N,baht,...,NaN,20240619.0,20240601,20240620,2024-06-21 12:37:25,2024-06-21 15:59:30.795783,TB4R001600,TVS GA ARPU,GA ARPU/RC,N


In [93]:
''' Generate Temp files '''

''' GROUP list '''
# grp_list_df = tmp_df[['METRIC_GRP', 'PRODUCT_GRP']].drop_duplicates().reset_index(drop=True)
# grp_list_df.dropna(how='all')
# # grp_list_df.dropna(axis=1, how='all')
# # grp_list_df.dropna(subset=['PRODUCT_GRP'])
grp_list_df = tmp_df.groupby(['METRIC_GRP', 'PRODUCT_GRP']).agg({'METRIC_CD': 'nunique', 'MIN_DAY': 'min', 'MAX_DAY': 'max'}).reset_index()
grp_list_df.rename(columns={'METRIC_CD': 'CNT_METRIC'}, inplace=True)
grp_list_df.to_excel(f'temp/Metric_Grp_List_{str_curr_dt}.xlsx', sheet_name='Data', index=False)
print(f'\n   -> Generate "Metric_Grp_List_{str_curr_dt}.xlsx" successfully')

''' METRIC list '''
# metric_list_df = tmp_df[['METRIC_GRP', 'PRODUCT_GRP', 'COMP_CD', 'METRIC_CD', 'METRIC_NAME']].drop_duplicates().reset_index(drop=True)
# metric_list_df.dropna(how='all')
metric_list_df = tmp_df.groupby(['METRIC_GRP', 'PRODUCT_GRP', 'COMP_CD', 'TMP_CD', 'TMP_NAME', 'AGG_TYPE', 'UOM']).agg({'CHANNEL_CD': 'nunique', 'ACTUAL_AS_OF': 'max', 'MIN_DAY': 'min', 'MAX_DAY': 'max'}).reset_index()
metric_list_df.rename(columns={'CHANNEL_CD': 'CNT_CHANNEL', 'ACTUAL_AS_OF': 'LAST_ACTUAL'}, inplace=True)
metric_list_df.to_excel(f'temp/Metric_Cd_List_{str_curr_dt}.xlsx', sheet_name='Data', index=False)
print(f'\n   -> Generate "Metric_Cd_List_{str_curr_dt}.xlsx" successfully')



   -> Generate "Metric_Grp_List_20240621.xlsx" successfully

   -> Generate "Metric_Cd_List_20240621.xlsx" successfully


In [98]:
''' Create Reconcile Data '''

rec_df = tmp_df

''' Filters '''
# rec_df = rec_df.loc[rec_df['TM_KEY_MTH']==202406]
# rec_df = rec_df.loc[rec_df['VIEW_GRP']=='']
rec_df = rec_df.loc[rec_df['CDS_FLAG']=='Y']
# rec_df = rec_df.loc[rec_df['CHANNEL_CD']=='ALL']
# rec_df = rec_df.loc[rec_df['COMP_CD']=='TRUE']
# rec_df = rec_df.loc[rec_df['PRODUCT_GRP']=='TOL']

# my_str = 'Prepaid Topping|Prepaid Pay per Use'
# my_str = '^Postpaid Revenue.*DTAC$'
# my_str = 'Inflow M1|Gross Add'
# my_str2 = 'CORP'
# my_str3 = ' B2C| B2B|Geo'

# rec_df = rec_df[rec_df['METRIC_CD']=='DB1R000900']
# rec_df = rec_df.loc[rec_df['METRIC_NAME'].str.contains(my_str)]
# rec_df = rec_df.loc[~rec_df['METRIC_NAME'].str.contains(my_str2)]
# rec_df = rec_df.loc[~rec_df['METRIC_NAME'].str.contains(my_str3)]
# rec_df = rec_df.replace(np.nan, None)
# rec_df = rec_df.fillna(0)
rec_df = rec_df.reset_index(drop=True)
rec_df.tail(3)

,TM_KEY_MTH,CENTER,METRIC_GRP,PRODUCT_GRP,COMP_CD,METRIC_CD,METRIC_NAME,CHANNEL_CD,AGG_TYPE,UOM,...,HH_TARGET,ACTUAL_AS_OF,MIN_DAY,MAX_DAY,PPN_TM,LOAD_DATE,TMP_CD,TMP_NAME,VIEW_GRP,CDS_FLAG
1311,202406,Revenue,Sales,TVS,TRUE,TB4S001400,TVS Now Gross Adds,ALL,S,subs,...,NaN,NaN,20240601,20240620,2024-06-21 12:37:25,2024-06-21 15:59:30.795783,TB4S001400,TVS Now Gross Adds,Gross Adds,Y
1312,202406,Sales,Sales,TOL,TRUE,TB3R000600,TOL Inflow M1 - Connected,ALL,S,baht,...,15321859.86,20240619.0,20240601,20240620,2024-06-21 12:37:25,2024-06-21 15:59:30.795783,TB3R000600,TOL Inflow M1 - Connected,Inflow M1,Y
1313,202406,Sales,Sales,TVS,TRUE,TB4R001000,TVS Inflow M1,ALL,S,baht,...,NaN,20240618.0,20240601,20240620,2024-06-21 12:37:25,2024-06-21 15:59:30.795783,TB4R001000,TVS Inflow M1,Inflow M1,Y


In [105]:
''' Aggregate '''

# Yearly
# agg_df = rec_df.groupby(['TM_KEY_YR', 'COMP_CD', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'CHANNEL_CD', 'UOM']).agg({'P_ACTUAL': 'sum', 'P_TARGET': 'sum', 'LOAD_DATE': 'max'}).reset_index()
# agg_df = rec_df.groupby(['TM_KEY_YR', 'COMP_CD', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'CHANNEL_CD', 'UOM']).agg({'P_ACTUAL': 'sum', 'G_ACTUAL': 'sum', 'H_ACTUAL': 'sum', 'HH_ACTUAL': 'sum', 'LOAD_DATE': 'max'}).reset_index()
# agg_df = rec_df.groupby(['TM_KEY_YR', 'COMP_CD', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'CHANNEL_CD', 'UOM']).agg({'P_ACTUAL': 'sum', 'G_ACTUAL': 'sum', 'H_ACTUAL': 'sum', 'HH_ACTUAL': 'sum', 'P_TARGET': 'sum', 'G_TARGET': 'sum', 'H_TARGET': 'sum', 'HH_TARGET': 'sum', 'LOAD_DATE': 'max'}).reset_index()

# Monthly
agg_df = rec_df.groupby(['TM_KEY_MTH', 'COMP_CD', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'CHANNEL_CD', 'UOM']).agg({'P_ACTUAL': 'sum', 'G_ACTUAL': 'sum', 'H_ACTUAL': 'sum', 'HH_ACTUAL': 'sum', 'P_TARGET': 'sum', 'G_TARGET': 'sum', 'H_TARGET': 'sum', 'HH_TARGET': 'sum', 'LOAD_DATE': 'max'}).reset_index()


''' Create Temp File '''
# agg_df.to_excel(f'temp/Temp.xlsx', sheet_name='Data', index=False)
# print(f'\n -> Generate "Temp.xlsx" successfully')

agg_df.tail()

,TM_KEY_MTH,COMP_CD,PRODUCT_GRP,METRIC_CD,METRIC_NAME,CHANNEL_CD,UOM,P_ACTUAL,G_ACTUAL,H_ACTUAL,HH_ACTUAL,P_TARGET,G_TARGET,H_TARGET,HH_TARGET,LOAD_DATE
1309,202406,TRUE,TVS,TB4S000100AH,TVS Gross Adds : Others,AH,subs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-06-21 15:59:30.795783
1310,202406,TRUE,TVS,TB4S000100AJ,TVS Gross Adds : Retail Sales,AJ,subs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-06-21 15:59:30.795783
1311,202406,TRUE,TVS,TB4S000104,TVS Gross Adds : TVS Now,ALL,subs,588.0,568.0,568.0,568.0,0.0,0.0,0.0,0.0,2024-06-21 15:59:30.795783
1312,202406,TRUE,TVS,TB4S001300,TVS Gross Adds (Install Location),ALL,subs,18.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,2024-06-21 15:59:30.795783
1313,202406,TRUE,TVS,TB4S001400,TVS Now Gross Adds,ALL,subs,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-06-21 15:59:30.795783


In [106]:
''' Pivot Table '''

# # Actual & Target
# pv_target = pd.pivot_table(x, values=['P_ACTUAL', 'P_TARGET'], index=['COMP_CD', 'TMP_CD', 'TMP_NAME'], columns='CHANNEL_CD', aggfunc='sum', fill_value=0)

# Actual
pv_actual = pd.pivot_table(tmp_df, values='P_ACTUAL', index=['COMP_CD', 'TMP_CD', 'TMP_NAME'], columns='CHANNEL_CD', aggfunc='sum', fill_value=0)
pv_actual['VERION'] = 'A'

# Target
pv_target = pd.pivot_table(tmp_df, values='P_TARGET', index=['COMP_CD', 'TMP_CD', 'TMP_NAME'], columns='CHANNEL_CD', aggfunc='sum', fill_value=0)
pv_target['VERION'] = 'T'

# Concat Dataframe
pd.concat([pv_actual, pv_target]).reset_index()

CHANNEL_CD,COMP_CD,TMP_CD,TMP_NAME,AA,AB,AC,AD,AE,AF,AG,AH,AI,AJ,AK,ALL,VERION
0,ALL,B0R000100,Total Revenue,0.00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,0.0,0.00,7.044772e+10,A
1,ALL,B0R00010001,Total Inflow M1,2971832.95,68016468.54,3.275207e+08,1.910183e+08,1.465521e+08,1.209969e+08,1.016870e+08,47300083.55,11036011.67,698438953.2,9758152.74,1.813195e+09,A
2,ALL,B0R00010001CORP,Total Inflow M1,0.00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,0.0,0.00,1.813195e+09,A
3,ALL,B0R00010002,Total Gross Adds,2407.00,145537.00,1.109452e+06,6.059240e+05,7.597180e+05,2.362990e+05,8.214460e+05,188723.00,17517.00,5039811.0,1479269.00,1.103859e+07,A
4,ALL,B0R0001002,%Revenue Growth (YTD YoY),0.00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,0.0,0.00,3.609000e+01,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,TRUE,VIN00042,Revenue (Corporate),0.00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,0.0,0.00,2.400000e+01,T
972,TRUE,VIN00043,Profitability (Corporate),0.00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,0.0,0.00,2.400000e+01,T
973,TRUE,VIN00050,Mobile Subs Share (Subs) : TMH,0.00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,0.0,0.00,0.000000e+00,T
974,TRUE,VIN00061,Prepaid Subs Share (Subs) : TMH,0.00,0.00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.00,0.0,0.00,0.000000e+00,T


## Generate Output file

In [45]:
# # to Excel file

# op_dir = 'temp'
# op_file = 'VINSIGHT Data Monitoring.xlsx'

# df.to_excel(f'{op_dir}/{op_file}', sheet_name='Data', index=False)
# print(f'\n  -> Generate "{op_file}" successfully')


 -> Generate "Metric_List.xlsx" successfully


In [44]:
# # to CSV file

# op_dir = 'temp'
# op_file = 'VINSIGHT Data Monitoring.csv'

# df.to_csv(f'{op_dir}/{op_file}', index=False, encoding='utf-8')
# print(f'\n  -> Generate "{op_file}" successfully')


 -> Generate "Metric_List.csv" successfully
